# RegNet Inference (submission generation)

In [7]:
import os
import sys
from pathlib import Path
import joblib
from joblib import Parallel, delayed
from tqdm import tqdm
import glob
import pandas as pd
import torch
import torch.nn as nn
import torchaudio
import numpy as np
from torchvision.models import regnet_y_800mf, RegNet_Y_800MF_Weights
import timm
import re
from torchaudio import functional as F_audio

In [8]:
## REUSE IN INFERENCE NOTEBOOK

custom_dataset_path = '/kaggle/input/birdclef2023-inference'
if os.path.exists(os.path.join(custom_dataset_path, 'utils.py')):
    sys.path.append(custom_dataset_path)
else:
    sys.path.append('..')
import utils

IS_IN_KAGGLE_ENV = utils.get_is_in_kaggle_env()

DATA_PATH = '/kaggle/input/birdclef-2023' if IS_IN_KAGGLE_ENV else '../data'
JOBLIB_PATH = custom_dataset_path if IS_IN_KAGGLE_ENV else './'

DEVICE = 'cpu'

AUDIO_LENGTH_S = 5
SAMPLE_RATE = 32_000

We are running code on Localhost


In [9]:
## REUSE IN INFERENCE NOTEBOOK

class BirdMelspecClf(nn.Module):
    def __init__(self, out_features, pretrained):
        super().__init__()
        
        # https://pytorch.org/vision/stable/models.html

        self.regnet = regnet_y_800mf(weights=RegNet_Y_800MF_Weights.DEFAULT) if pretrained else regnet_y_800mf()

        """
        Original:
        RegnetCNN(
        (regnet): RegNet(
            (stem): SimpleStemIN(
            (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
        )"""
        self.regnet.stem = nn.Sequential(
            nn.Conv2d(2, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),
        )
        
        # Fine-tune the regnet classifier
        self.regnet.fc = nn.Linear(self.regnet.fc.in_features, out_features)

        self.softmax = nn.Softmax(dim=1)
 
    def forward(self, x):
        logits = self.regnet(x)
        probas = self.softmax(logits)

        return logits, probas


def get_model(out_features, device, pretrained=False, load_state_dict=True, state_dict_starts_with=f"{AUDIO_LENGTH_S}s_regnetY800MF_"):
    model = BirdMelspecClf(out_features=out_features, pretrained=pretrained)
    print(f"Loaded model {model.__class__.__name__} with {sum(p.numel() for p in model.parameters())} parameters, pretained={pretrained}")
    model.to(device)

    if not load_state_dict:
        return model

    model_files = [f for f in os.listdir(JOBLIB_PATH) if f.startswith(state_dict_starts_with) and f.endswith('.pt')]
    if len(model_files) == 0:
        print(f"No model starting with {state_dict_starts_with} found in {JOBLIB_PATH}")
        return model
    
    # Extract timestamp from the filenames and sort based on it
    model_files.sort(key=lambda x: int(re.findall(r'\d+', x)[-1]) if re.findall(r'\d+', x) else -1)

    # The latest model file is the last one in the sorted list
    latest_model_file = model_files[-1]
    model_path = os.path.join(JOBLIB_PATH, latest_model_file)
    model.load_state_dict(torch.load(model_path))
    print(f"Loaded model weights from {model_path}")
    model.to(device)

    return model


def get_label_encoder():
    label_encoder_path = os.path.join(JOBLIB_PATH, 'label_encoder.joblib')
    label_encoder = joblib.load(label_encoder_path)
    print(f"Loaded label encoder from {label_encoder_path}")
    return label_encoder

In [10]:
## REUSE IN INFERENCE NOTEBOOK

def resample(audio, current_sample_rate, desired_sample_rate=SAMPLE_RATE):
    resampler = torchaudio.transforms.Resample(orig_freq=current_sample_rate, new_freq=desired_sample_rate)
    resampled_audio = resampler(audio)
    return resampled_audio

def load_audio(audio_path, sample_rate=SAMPLE_RATE):
    audio, sr = torchaudio.load(audio_path)
    if sr != sample_rate:
        audio = resample(audio, sr, sample_rate)
    return audio

# Using librosa defaults for n_fft and hop_length
def get_melspec_transform(sample_rate=SAMPLE_RATE, n_fft=2048, hop_length=512, n_mels=128):
    return torchaudio.transforms.MelSpectrogram(
        sample_rate=sample_rate,
        n_fft=n_fft,
        hop_length=hop_length,
        n_mels=n_mels,
    )

# Using librosa defaults for top_db
def get_melspec_db_transform(stype='power', top_db=80):
    return torchaudio.transforms.AmplitudeToDB(
        stype=stype,
        top_db=top_db
    )

# Copied from torchaudio/transforms/_transforms.py (to avoid converting to melspec twice)
dct_mat = F_audio.create_dct(128, 128, "ortho")
def get_mfcc_from_melspec(melspec):
    return torch.matmul(melspec.transpose(-1, -2), dct_mat).transpose(-1, -2)

def normalize_tensor(tensor):
    min_val = torch.min(tensor)
    max_val = torch.max(tensor)
    if max_val - min_val == 0:
        return tensor
    else:
        return (tensor - min_val) / (max_val - min_val)

In [11]:
filepaths = glob.glob(f"{DATA_PATH}/test_soundscapes/*.ogg")
print(f"filepaths length: {len(filepaths)} (amount of test soundscapes)")

filepaths length: 1 (amount of test soundscapes)


# Inference

In [12]:
debug = False
simulate_200_files = False

if simulate_200_files:
    filepaths = [filepaths[0] for i in range(200)] # simulate submission
    print(f"filepaths length: {len(filepaths)} after simulation additions")

label_encoder = get_label_encoder()
model = get_model(out_features=len(label_encoder.classes_), device=DEVICE, pretrained=False, load_state_dict=True)
model.eval()

MIN_WINDOW = AUDIO_LENGTH_S * SAMPLE_RATE
melspec_transform = get_melspec_transform(n_mels=128)
melspec_db_transform = get_melspec_db_transform()

def infer(filepath):
    all_predictions = []
    name = Path(filepath).stem
    audio = load_audio(filepath)
    audio_len_s = audio.shape[1] / SAMPLE_RATE
    debug and print(f"Infering file {filepath} with length {audio_len_s} s")
    n_crops = int(audio_len_s // 5)
    for i in range(n_crops):
        debug and print(f"Crop {i} / {n_crops}")
        debug and print(f"Audio length: {len(audio)}")
        crop = audio[:, i*MIN_WINDOW:(i+1)*MIN_WINDOW]
        debug and print(f"Crop dimensions: {crop.shape}")
        melspec = melspec_db_transform(melspec_transform(crop))
        norm_melspec = normalize_tensor(melspec)
        mfcc = get_mfcc_from_melspec(norm_melspec)
        norm_mfcc = normalize_tensor(mfcc)
        features = torch.cat((norm_melspec, norm_mfcc), dim=0)
        debug and print(f"features shape: {features.shape}") # [2, 128, 313]
        features = features.unsqueeze(0) # add batch dimension (1)
        debug and print(f"features unsqueezed shape: {features.shape}") # [1, 2, 128, 313]
        with torch.no_grad():
            logit, proba = model(features)
        t = (i + 1) * 5
        all_predictions.append({"row_id": f'{name}_{t}',"predictions": proba})
        debug and print('---')
    return all_predictions

if debug:
    all_preds = []
    for filepath in tqdm(filepaths, desc='Infering files'):
        all_preds.append(infer(filepath))
else:
    parallel_task = (delayed(infer)(filepath) for filepath in tqdm(filepaths, desc='Infering files'))
    all_preds = Parallel(n_jobs=os.cpu_count())(parallel_task)

all_preds_flat = [item for sublist in all_preds for item in sublist]

print(f"all_preds length: {len(all_preds)}, all_preds_flat length: {len(all_preds_flat)}")

Loaded label encoder from ./label_encoder.joblib
Loaded model BirdMelspecClf with 5834839 parameters, pretained=False
Loaded model weights from ./5s_regnetY800MF_hailmary_e82_valacc66_traacc81_1684949946082.pt


Infering files: 100%|██████████| 1/1 [00:00<00:00, 855.28it/s]


all_preds length: 1, all_preds_flat length: 120


In [13]:
all_preds_flat[100]['predictions'][0]

tensor([3.5873e-03, 9.8154e-03, 6.9927e-03, 4.1697e-04, 1.3031e-03, 8.2832e-04,
        3.3516e-02, 2.0091e-03, 6.6309e-03, 2.3998e-04, 1.9321e-03, 1.2504e-03,
        1.1416e-03, 2.1074e-02, 1.9221e-03, 1.0261e-03, 8.7702e-04, 8.7235e-03,
        5.9264e-03, 6.6665e-04, 7.7649e-04, 2.0435e-04, 1.0143e-03, 4.0895e-04,
        3.6320e-03, 2.7631e-03, 7.7882e-04, 7.4817e-03, 3.0988e-03, 6.0705e-04,
        1.0473e-02, 1.4140e-03, 6.4480e-04, 4.1580e-03, 4.0434e-03, 6.5054e-04,
        3.9136e-04, 6.5865e-04, 1.7866e-03, 3.5598e-03, 8.0934e-03, 5.9275e-04,
        1.3937e-03, 6.7864e-04, 4.9295e-04, 8.7874e-04, 2.6739e-03, 1.1941e-03,
        1.2637e-03, 5.9968e-03, 1.9771e-03, 6.2765e-04, 8.0388e-04, 1.8256e-02,
        6.6349e-04, 1.0096e-03, 1.7688e-03, 2.3336e-03, 1.8552e-03, 3.6821e-04,
        1.4258e-03, 5.4331e-02, 3.5175e-04, 5.7271e-04, 2.5225e-03, 1.0602e-03,
        7.2389e-03, 3.3240e-03, 7.2323e-04, 1.1509e-03, 3.6170e-03, 5.0559e-03,
        7.0763e-04, 8.2847e-04, 4.2156e-

In [14]:
df = pd.concat([
    pd.DataFrame({'row_id': [p['row_id'] for p in all_preds_flat]}), 
    pd.DataFrame(torch.stack([p['predictions'][0] for p in all_preds_flat]).numpy(), columns=label_encoder.classes_)
], axis=1)

df

,row_id,abethr1,abhori1,abythr1,afbfly1,afdfly1,afecuc1,affeag1,afgfly1,afghor1,...,yebsto1,yeccan1,yefcan,yelbis1,yenspu1,yertin1,yesbar1,yespet1,yetgre1,yewgre1
0,soundscape_29201_5,0.007974,0.005609,0.018319,0.000835,0.000797,0.007523,0.002114,0.000260,0.018849,...,0.000412,0.000361,0.001426,0.000679,0.000352,0.001149,0.000755,0.000241,0.000721,0.001485
1,soundscape_29201_10,0.007716,0.019926,0.007388,0.000827,0.004440,0.005641,0.014039,0.001206,0.009620,...,0.000659,0.001477,0.003734,0.001844,0.002038,0.007202,0.004793,0.002468,0.000673,0.003918
2,soundscape_29201_15,0.031356,0.011575,0.003777,0.000106,0.000283,0.023654,0.000719,0.000114,0.002021,...,0.000244,0.000255,0.001712,0.000722,0.000210,0.000123,0.000090,0.007458,0.000149,0.000805
3,soundscape_29201_20,0.000365,0.001087,0.000502,0.001627,0.000390,0.000208,0.000693,0.000123,0.001634,...,0.000258,0.010774,0.001461,0.000793,0.000159,0.000234,0.000353,0.000199,0.000133,0.000194
4,soundscape_29201_25,0.002430,0.003676,0.005158,0.001155,0.000678,0.000918,0.001302,0.001081,0.006501,...,0.000666,0.008194,0.003169,0.000878,0.000444,0.000650,0.000203,0.003794,0.000485,0.000636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,soundscape_29201_580,0.001121,0.003618,0.000683,0.000787,0.000980,0.000204,0.001614,0.000533,0.001002,...,0.000173,0.008112,0.002921,0.000396,0.000276,0.000991,0.000444,0.000779,0.000111,0.000369
116,soundscape_29201_585,0.003116,0.028437,0.003665,0.000215,0.001007,0.001214,0.352138,0.000463,0.001656,...,0.000114,0.000396,0.000318,0.000108,0.000342,0.020242,0.000822,0.000060,0.000034,0.000338
117,soundscape_29201_590,0.001505,0.014158,0.004417,0.001024,0.003786,0.000953,0.026137,0.001610,0.008973,...,0.000556,0.002121,0.005248,0.001279,0.000914,0.015486,0.003423,0.001444,0.000412,0.001056
118,soundscape_29201_595,0.002845,0.001743,0.002154,0.000598,0.002937,0.000260,0.002040,0.001834,0.001661,...,0.000322,0.003490,0.004431,0.001672,0.000656,0.002247,0.001400,0.001381,0.000189,0.000579


In [15]:
df.to_csv('submission.csv', index=False)

In [16]:
!ls submission.csv

submission.csv
